In [1]:
import os
import glob
import json
import numpy as np
import pandas as pd
import geopandas as gpd
import clima_anom as ca

import matplotlib.cbook
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec

import cartopy
import cartopy.feature as cf
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import cartopy.io.shapereader as shpreader

In [2]:
method = 'pearson'

In [3]:
with open ('../../json/directories.json') as f:
    DIRS = json.load(f)

In [4]:
if os.environ['COMPUTERNAME'] == 'DELL_LAPTOP':
    export_dir = 'D:/Data/PF/ENSEMBLE/ENSEMBLE_SA/' + method + '/'
    PF_ensemble_files = 'D:\Data\PF\ENSEMBLE\list\TRMM_GPM_summer_200012_2021_02.csv'
else:
    export_dir = '/mnt/Data/Data/PF/ENSEMBLE/ENSEMBLE_SA/' + method + '/'

print(os.environ['COMPUTERNAME'])
print(f'Export dir: {export_dir}')

DELL_LAPTOP
Export dir: D:/Data/PF/ENSEMBLE/ENSEMBLE_SA/pearson/


In [5]:
if method == 'pearson':
    comms_file = 'Comm_IMERG_05x05_pearson_summer_noc'
else:
    print('ERROR')

In [6]:
file_list = pd.read_csv(PF_ensemble_files)
# file_list = pd.read_csv(DIRS['dir_ENSEMBLE_list']+'TRMM_GPM_summer_200012_2021_02.csv')
len_files = len(file_list)
print(f'Files Number: {len_files}')
print(file_list['file_name'].iloc[0])
print(file_list['file_name'].iloc[-1])

Files Number: 63
/mnt/Data/Data/PF/TRMM/rpf_SA/rpf_200012_level2.csv
/mnt/Data/Data/PF/GPM/rgpf_SA/rgpf_202102_level2.csv


In [19]:
communities_poligon = gpd.read_file(DIRS['dir_comm'] + comms_file + '.geojson')
communities_index = communities_poligon['Comm'].values.astype(int)
communities_index

array([ 10,  15,  20,  25,  30,  35,  40,  45,  50,  55,  60,  65,  70,
        75,  80,  90,  95, 100])

In [8]:
Comunidad_file = DIRS['dir_comm'] + comms_file + '.shp'
Comunidades = gpd.read_file(Comunidad_file)
num_comm = len(Comunidades)
print(f'Number of Communities: {num_comm}')

Number of Communities: 18


In [20]:
# test names
comm_id = communities_index[0]
comm_name = comms_file + '_C'+str(comm_id).zfill(3)
salida = export_dir + comm_name + '.csv'
salida

'D:/Data/PF/ENSEMBLE/ENSEMBLE_SA/pearson/Comm_IMERG_05x05_pearson_summer_noc_C010.csv'

In [22]:
for comm_id in range(num_comm):

    comm_id_new = communities_index[comm_id]
    print(f'Community: {comm_id}')

    for nn in range(len_files):

        if os.environ['COMPUTERNAME'] == 'DELL_LAPTOP':
            data = pd.read_csv(file_list['file_name'][nn].replace('/mnt/Data/Data/PF/','D:/Data/PF/'))
        else:
            data = pd.read_csv(file_list['file_name'][nn], sep=',')

        data[data==-88.88] = np.nan
        data[data==-99.99] = np.nan

        PF_GPD = gpd.GeoDataFrame(data[data.columns[2:]], geometry=gpd.points_from_xy(data['lon'],data['lat'], crs="EPSG:4326"))
        num_points = len(PF_GPD['geometry']) 

        points_interception = [] 

        for t in range(num_points):

            if Comunidades['geometry'][comm_id].contains(PF_GPD['geometry'][t]) == True:
                points_interception.append(t)

        dic_tmp = data.loc[points_interception]
        dic_tmp = dic_tmp.reset_index(drop=True)

        if nn == 0:
            dict_out = dic_tmp

        else:
            dict_out = pd.concat([dic_tmp, dict_out], axis=0)

        # print(f'Points in the community: {len(dic_tmp)}')

    comm_name = comms_file + '_C'+str(comm_id_new).zfill(3)
    salida = export_dir + comm_name + '.csv'
    print(f'Exporting data to {salida}')
    dict_out.to_csv(salida,header=True,index=False)

Community: 0
Exporting data to D:/Data/PF/ENSEMBLE/ENSEMBLE_SA/pearson/Comm_IMERG_05x05_pearson_summer_noc_C010.csv
Community: 1
Exporting data to D:/Data/PF/ENSEMBLE/ENSEMBLE_SA/pearson/Comm_IMERG_05x05_pearson_summer_noc_C015.csv
Community: 2
Exporting data to D:/Data/PF/ENSEMBLE/ENSEMBLE_SA/pearson/Comm_IMERG_05x05_pearson_summer_noc_C020.csv
Community: 3
Exporting data to D:/Data/PF/ENSEMBLE/ENSEMBLE_SA/pearson/Comm_IMERG_05x05_pearson_summer_noc_C025.csv
Community: 4
Exporting data to D:/Data/PF/ENSEMBLE/ENSEMBLE_SA/pearson/Comm_IMERG_05x05_pearson_summer_noc_C030.csv
Community: 5
Exporting data to D:/Data/PF/ENSEMBLE/ENSEMBLE_SA/pearson/Comm_IMERG_05x05_pearson_summer_noc_C035.csv
Community: 6
Exporting data to D:/Data/PF/ENSEMBLE/ENSEMBLE_SA/pearson/Comm_IMERG_05x05_pearson_summer_noc_C040.csv
Community: 7
Exporting data to D:/Data/PF/ENSEMBLE/ENSEMBLE_SA/pearson/Comm_IMERG_05x05_pearson_summer_noc_C045.csv
Community: 8
Exporting data to D:/Data/PF/ENSEMBLE/ENSEMBLE_SA/pearson/C

In [68]:
# Comunidad_file = 'Comm_IMERG_05x05_sync_summer_P90_tau3_SP95'
# comm_id = 1
# comm_name = 'C'+str(comm_id+1).zfill(2)
# print(comm_name)

# levels = np.arange(0.5,20.5,0.5)

# for nn in range(63):

#     data = pd.read_csv(file_list['file_name'][nn], sep=',')
#     data[data==-88.88] = np.nan
#     data[data==-99.99] = np.nan

#     PF_GPD = gpd.GeoDataFrame(data[data.columns[2:]], geometry=gpd.points_from_xy(data['lon'],data['lat'], crs="EPSG:4326"))

#     comm_GPD = gpd.read_file(DIRS['dir_communities']'+Comunidad_file+'.shp')

#     num_comm = len(comm_GPD['geometry'])
#     num_points = len(PF_GPD['geometry']) 

#     points_interception = []  

#     for t in range(num_points):

#         if comm_GPD['geometry'][comm_id].contains(PF_GPD['geometry'][t]) == True:
#             points_interception.append(t)

#     dic_tmp = data.loc[points_interception]
#     dic_tmp = dic_tmp.reset_index(drop=True)

#     if nn == 0:
#         dict_out = dic_tmp

#     else:
#         dict_out = pd.concat([dic_tmp, dict_out], axis=0)

#     print(f'Points in the community: {len(dic_tmp)}')

# # export dataframe to csv
# salida = '/mnt/Data/Data/PF/ENSEMBLE/ENSEMBLE_SA/'+Comunidad_file+'_'+comm_name+'.csv'
# print(f'Exporting data to {salida}')
# dict_out.to_csv(salida,header=True,index=False)